**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

```Epsilon``` can be seen as the probability to select a random action. It is essential to learn a policy, as it ensures the exploration part is not forgotten, in order to obtain more information on the environment. Otherwise, the algorithm risks to only exploit its known information, neglecting to ensure it is the optimal behaviour.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1 #modif

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1 #modif
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [17]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=100 # set small when debugging
epochs_test=100 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The ```position``` and ```board``` represent the environment. The ```position``` indicates where the rat is on the grid with the value ```1``` and the limit of the grid with the value ```-1```. The ```board``` indicates the positions of the remaining cheese or poison, giving at the same time the corresponding reward associated. The rat only sees a part of this environment : the ```state```.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [18]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        a = np.random.randint(0, 4)
        return(a)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [19]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [20]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 7.0/8.0. Average score (-1.0)
Win/lose count 8.0/13.0. Average score (-3.0)
Win/lose count 4.0/9.0. Average score (-3.6666666666666665)
Win/lose count 11.0/6.0. Average score (-1.5)
Win/lose count 14.5/5.0. Average score (0.7)
Win/lose count 8.0/12.0. Average score (-0.08333333333333333)
Win/lose count 12.0/13.0. Average score (-0.21428571428571427)
Win/lose count 8.5/18.0. Average score (-1.375)
Win/lose count 7.5/15.0. Average score (-2.0555555555555554)
Win/lose count 6.0/14.0. Average score (-2.65)
Win/lose count 11.5/17.0. Average score (-2.909090909090909)
Win/lose count 12.0/22.0. Average score (-3.5)
Win/lose count 4.5/8.0. Average score (-3.5)
Win/lose count 10.0/6.0. Average score (-2.9642857142857144)
Win/lose count 9.0/17.0. Average score (-3.3)
Win/lose count 13.0/25.0. Average score (-3.84375)
Win/lose count 8.0/7.0. Average score (-3.5588235294117645)
Win/lose count 9.0/16.0. Average score (-3.75)
Win/lose count 9.5/12.0. Average score (-3.6842105263157894

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




> From the definition of the $Q^\pi$ function :
> $Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t=0}^{T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] $
>
> Then
> $Q^\pi(s,a)=E_{p^{\pi}}[r(s,a)+\sum_{t=1}^{T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] $
> 
> $Q^\pi(s,a)=E_{p^{\pi}}[r(s,a)+\sum_{t=0}^{T-1}\gamma^{t+1}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a] $
>
> $Q^\pi(s,a)=E_{p^{\pi}}[r(s,a)+\gamma\sum_{t=0}^{T-1}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a] $
> 
> By defining $(s',a')$ such as, for $t\geq0$, $(s'_{t},a'_{t}) = (s_{t+1},a_{t+1})$, we obtain the expected result:
> 
> $ Q^{\pi}(s,a)=E_{(s',a')\sim p^{\pi}(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] $
>
> If $\pi^*$ is optimal, then we know that : 
> 
> $ Q^*(s,a)=\max_{\pi}Q^\pi(s,a)$ and $ \pi^*(a|s)= \left\{
    \begin{array}{ll}
        1 & \mbox{if } a =\underset{a'}{\arg\max} Q^*(s,a') \\
        0 & \mbox{else.}
    \end{array}
\right.$
> 
> Therefore, $ Q^*(s,a)=E_{(s',a')\sim \pi^*(.|s,a)}[r(s,a)+\gamma Q^*(s',a')] $
> 
> $ Q^*(s,a)=E_{(s')\sim \pi^*(.|s,a)}[r(s,a)+\max_{a'}\gamma Q^*(s',a')] $
> 
> 
> As a consequence, trying to minimize the squared difference between our Q-function and this optimal form seems a plausible objective, and gives us:
>
> $ \mathcal{L}(\theta) = \Vert Q^*(s,a,\theta)-Q(s,a,\theta)\Vert^{2}$
>
> $ \mathcal{L}(\theta) = \Vert E_{(s',)\sim \pi^*(.|s,a)}[r+\max_{a'}\gamma Q^{\pi}(s',a',\theta)]-Q(s,a,\theta)\Vert^{2}$
>
> $ \mathcal{L}(\theta) = E_{(s',)\sim \pi^*(.|s,a)} \Vert r+\max_{a'}\gamma Q^{\pi}(s',a',\theta)-Q(s,a,\theta)\Vert^{2}$

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [21]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory)==self.max_memory:
            self.memory=self.memory[1:]+[m]
        else:
            self.memory=self.memory+[m]


    def random_access(self):
        rand = np.random.randint(0, len(self.memory))
        return self.memory[rand]

***
The pipeline we will use for training is given below:

In [22]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [44]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        best_a=np.argmax(self.model.predict(np.expand_dims(s, 0))[0])
        return best_a

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_i, n_s_i, a_i, r_i, game_over_i = self.memory.random_access()
            
            input_states[i]=s_i.copy()
            
            if game_over_i:
                target_q[i,a_i]=r_i
            else:
                n_target= self.model.predict(np.expand_dims(n_s_i, 0))[0]
                target_q[i,a_i] = r_i + self.discount*max(n_target)-target_q[i,a_i]

        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Flatten(input_shape=(5, 5, self.n_state)))
        model.add(Dense(32, activation="relu"))
        model.add(Dense(4, activation=None))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [81]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 64)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train0.mp4'))

Epoch 000/100 | Loss 0.0097 | Win/lose count 4.5/8.0 (-3.5)
Epoch 001/100 | Loss 0.0155 | Win/lose count 5.0/2.0 (3.0)
Epoch 002/100 | Loss 0.0149 | Win/lose count 5.0/6.0 (-1.0)
Epoch 003/100 | Loss 0.0116 | Win/lose count 9.5/3.0 (6.5)
Epoch 004/100 | Loss 0.0069 | Win/lose count 5.0/3.0 (2.0)
Epoch 005/100 | Loss 0.0121 | Win/lose count 7.0/7.0 (0.0)
Epoch 006/100 | Loss 0.0091 | Win/lose count 2.5/0 (2.5)
Epoch 007/100 | Loss 0.0115 | Win/lose count 11.0/9.0 (2.0)
Epoch 008/100 | Loss 0.0169 | Win/lose count 3.0/7.0 (-4.0)
Epoch 009/100 | Loss 0.0172 | Win/lose count 2.5/3.0 (-0.5)
Epoch 010/100 | Loss 0.0110 | Win/lose count 11.0/6.0 (5.0)
Epoch 011/100 | Loss 0.0106 | Win/lose count 2.5/3.0 (-0.5)
Epoch 012/100 | Loss 0.0199 | Win/lose count 6.5/2.0 (4.5)
Epoch 013/100 | Loss 0.0040 | Win/lose count 2.5/1.0 (1.5)
Epoch 014/100 | Loss 0.0060 | Win/lose count 2.0/1.0 (1.0)
Epoch 015/100 | Loss 0.0075 | Win/lose count 0.5/1.0 (-0.5)
Epoch 016/100 | Loss 0.0105 | Win/lose count 5.0/4

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [82]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        model = Sequential()
        model.add(Conv2D(filters=64, kernel_size=2, activation="relu", input_shape=(5, 5, self.n_state)))
        model.add(Conv2D(filters=32, kernel_size=2, activation="relu"))
        model.add(Flatten())
        model.add(Dense(4, activation=None))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [83]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.2, memory_size=2000, batch_size = 64)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train0.mp4'))

Epoch 000/100 | Loss 0.0046 | Win/lose count 3.5/2.0 (1.5)
Epoch 001/100 | Loss 0.0081 | Win/lose count 6.5/7.0 (-0.5)
Epoch 002/100 | Loss 0.0119 | Win/lose count 11.0/7.0 (4.0)
Epoch 003/100 | Loss 0.0075 | Win/lose count 4.5/4.0 (0.5)
Epoch 004/100 | Loss 0.0062 | Win/lose count 6.5/3.0 (3.5)
Epoch 005/100 | Loss 0.0036 | Win/lose count 2.5/1.0 (1.5)
Epoch 006/100 | Loss 0.0078 | Win/lose count 2.5/4.0 (-1.5)
Epoch 007/100 | Loss 0.0115 | Win/lose count 11.5/9.0 (2.5)
Epoch 008/100 | Loss 0.0123 | Win/lose count 4.5/3.0 (1.5)
Epoch 009/100 | Loss 0.0098 | Win/lose count 2.0/3.0 (-1.0)
Epoch 010/100 | Loss 0.0045 | Win/lose count 6.5/7.0 (-0.5)
Epoch 011/100 | Loss 0.0092 | Win/lose count 2.5/5.0 (-2.5)
Epoch 012/100 | Loss 0.0082 | Win/lose count 5.5/9.0 (-3.5)
Epoch 013/100 | Loss 0.0068 | Win/lose count 0.5/1.0 (-0.5)
Epoch 014/100 | Loss 0.0040 | Win/lose count 4.5/0 (4.5)
Epoch 015/100 | Loss 0.0099 | Win/lose count 5.5/5.0 (0.5)
Epoch 016/100 | Loss 0.0027 | Win/lose count 7.5/

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [89]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 64)
agent_cnn.load(name_weights='cnn_trainmodel_t03e01.h5',name_model='cnn_trainmodel_t03e01.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.2, memory_size=2000, batch_size = 64)
agent_fc.load(name_weights='fc_trainmodel_t03e02.h5',name_model='fc_trainmodel_t03e02.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 17.5/4.0. Average score (13.5)
Win/lose count 18.5/7.0. Average score (12.5)
Win/lose count 7.0/0. Average score (10.666666666666666)
Win/lose count 18.0/3.0. Average score (11.75)
Win/lose count 12.5/3.0. Average score (11.3)
Win/lose count 13.5/4.0. Average score (11.0)
Win/lose count 22.5/8.0. Average score (11.5)
Win/lose count 11.5/8.0. Average score (10.5)
Win/lose count 8.5/3.0. Average score (9.944444444444445)
Win/lose count 17.0/5.0. Average score (10.15)
Win/lose count 18.5/5.0. Average score (10.454545454545455)
Win/lose count 15.0/5.0. Average score (10.416666666666666)
Win/lose count 16.0/2.0. Average score (10.692307692307692)
Win/lose count 11.0/4.0. Average score (10.428571428571429)
Win/lose count 22.5/3.0. Average score (11.033333333333333)
Win/lose count 20.0/8.0. Average score (11.09375)
Win/lose count 26.5/9.0. Average score (11.470588235294118)
Win/lose count 21.5/5.0. Average score (11.75)
Win/lose count 27.5/7.0. Average score (12

Win/lose count 14.5/3.0. Average score (7.423913043478261)
Win/lose count 13.0/8.0. Average score (7.372340425531915)
Win/lose count 12.0/8.0. Average score (7.302083333333333)
Win/lose count 6.0/3.0. Average score (7.214285714285714)
Win/lose count 10.5/6.0. Average score (7.16)
Win/lose count 12.0/3.0. Average score (7.196078431372549)
Win/lose count 8.0/3.0. Average score (7.153846153846154)
Win/lose count 15.5/8.0. Average score (7.160377358490566)
Win/lose count 14.0/6.0. Average score (7.175925925925926)
Win/lose count 6.5/6.0. Average score (7.054545454545455)
Win/lose count 16.5/9.0. Average score (7.0625)
Win/lose count 15.0/4.0. Average score (7.131578947368421)
Win/lose count 6.0/2.0. Average score (7.077586206896552)
Win/lose count 13.0/2.0. Average score (7.1440677966101696)
Win/lose count 9.5/4.0. Average score (7.116666666666666)
Win/lose count 11.0/7.0. Average score (7.065573770491803)
Win/lose count 5.0/5.0. Average score (6.951612903225806)
Win/lose count 19.0/6.0. A

In [85]:
HTML(display_videos('cnn_test10.mp4'))

In [86]:
HTML(display_videos('fc_test10.mp4'))

The CNN model tends to get better results than the fully-connected one, especially with a low epsilon (T=0.7, epsilon=0.1 : FC=15.9; CNN=35.1; T=0.7, epsilon=0.2 : FC=37.4; CNN = 38.3; T=0.3, epsilon=0.2 : FC=6.6; CNN=11.7; T=0.3, epsilon=0.1 : FC=3.6; CNN=12.2). The main problem is the incomplete exploration of the map : the rat tends to move back and forth between already visited positions, especially when there are no bonus visible. The ```epsilon``` value may help: if it tends to zero, there is nearly no exploration of the map, but it is not sufficient, as it will also push the rat to go towards maluses if it is too high, as it introduces random behaviour. 
In addition, if the temperature is too low, it is more difficult to achieve a satisfying training, as the rat will more easily be in a position in which it doesn't see any malus.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [62]:
def train_explore(agent,env,epoch,prefix='',eps_d=0.8):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        agent.set_epsilon(agent.epsilon*(eps_d))
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
        reward=0
        if train:
            reward = -self.malus_position[self.x, self.y]
            self.malus_position[self.x, self.y] = 0.1
        
        reward = reward + self.board[self.x, self.y]

        self.t = self.t + 1

        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    
### use those samples of code:
#In train explore:
# state, reward, game_over = env.act(action, train=True)

# ## In Environment exploring:
# # You will have to change n_state to 3 because you will use one more layer!
# reward = 0
# if train:
#     reward = -self.malus_position[self.x, self.y]
# self.malus_position[self.x, self.y] = 0.1

# reward = reward + self.board[self.x, self.y]
# # 3 "feature" states instead of 2
# state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
#                                 self.board.reshape(self.grid_size, self.grid_size,1),
#                         self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)###

In [66]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.5, memory_size=2000, batch_size = 64,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/100 | Loss 0.0145 | Win/lose count 5.0/24.700000000000085 (-19.700000000000085)
Epoch 001/100 | Loss 0.0137 | Win/lose count 11.899999999999999/25.900000000000063 (-14.000000000000064)
Epoch 002/100 | Loss 0.0219 | Win/lose count 4.8/25.50000000000006 (-20.70000000000006)
Epoch 003/100 | Loss 0.0106 | Win/lose count 8.200000000000001/24.800000000000054 (-16.60000000000005)
Epoch 004/100 | Loss 0.0175 | Win/lose count 15.000000000000004/22.000000000000025 (-7.000000000000021)
Epoch 005/100 | Loss 0.0204 | Win/lose count 4.1/24.100000000000037 (-20.000000000000036)
Epoch 006/100 | Loss 0.0164 | Win/lose count 10.400000000000004/19.49999999999998 (-9.099999999999975)
Epoch 007/100 | Loss 0.0108 | Win/lose count 4.8/22.900000000000034 (-18.100000000000033)
Epoch 008/100 | Loss 0.0100 | Win/lose count 5.500000000000001/21.200000000000017 (-15.700000000000017)
Epoch 009/100 | Loss 0.0145 | Win/lose count 13.000000000000007/21.800000000000004 (-8.799999999999997)
Epoch 010/100 | Los

Epoch 082/100 | Loss 0.0184 | Win/lose count 18.799999999999997/18.4 (0.3999999999999986)
Epoch 083/100 | Loss 0.0100 | Win/lose count 19.999999999999993/22.100000000000062 (-2.100000000000069)
Epoch 084/100 | Loss 0.0154 | Win/lose count 21.19999999999999/18.800000000000008 (2.399999999999981)
Epoch 085/100 | Loss 0.0104 | Win/lose count 12.400000000000006/20.000000000000018 (-7.600000000000012)
Epoch 086/100 | Loss 0.0110 | Win/lose count 18.799999999999997/20.40000000000003 (-1.6000000000000334)
Epoch 087/100 | Loss 0.0167 | Win/lose count 10.000000000000004/19.60000000000001 (-9.600000000000005)
Epoch 088/100 | Loss 0.0178 | Win/lose count 8.400000000000002/19.0 (-10.599999999999998)
Epoch 089/100 | Loss 0.0122 | Win/lose count 10.400000000000004/25.500000000000057 (-15.100000000000053)
Epoch 090/100 | Loss 0.0107 | Win/lose count 3.9999999999999996/21.10000000000003 (-17.10000000000003)
Epoch 091/100 | Loss 0.0124 | Win/lose count 14.400000000000007/32.50000000000014 (-18.10000000

In [71]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 14.000000000000007/18.599999999999998. Average score (-4.599999999999991)
Win/lose count 15.200000000000008/21.300000000000047. Average score (-5.350000000000015)
Win/lose count 11.600000000000005/20.200000000000024. Average score (-6.433333333333349)
Win/lose count 13.600000000000007/25.70000000000012. Average score (-7.8500000000000405)
Win/lose count 18.0/21.600000000000055. Average score (-7.000000000000043)
Win/lose count 8.000000000000002/18.099999999999987. Average score (-7.5166666666667)
Win/lose count 8.000000000000002/22.10000000000005. Average score (-8.457142857142893)
Win/lose count 7.200000000000002/19.300000000000004. Average score (-8.912500000000032)
Win/lose count 18.4/21.50000000000005. Average score (-8.266666666666701)
Win/lose count 14.800000000000008/21.40000000000005. Average score (-8.100000000000033)
Win/lose count 17.200000000000003/18.800000000000004. Average score (-7.50909090909094)
Win/lose count 22.799999999999983/25.400000000000073. Aver

Win/lose count 8.000000000000002/19.1. Average score (-8.074226804123729)
Win/lose count 14.400000000000007/20.50000000000003. Average score (-8.054081632653078)
Win/lose count 13.600000000000007/21.70000000000005. Average score (-8.05454545454547)
Win/lose count 8.000000000000002/18.099999999999987. Average score (-8.075000000000017)
Final score: -8.075000000000017


We can see that the behaviour is rapidly much more effective than in the previous training : the rat does not visit the same positions more than once, at least during the first part of the exploration. In some cases, the rat manages to get nearly all the bonuses, eventhough in other, it only reaches a part of it before getting stuck by the fact that it does not see any position without maluses (already visited or poisonous) anymore.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***